In [2]:
#penalized cox model to predict factors influencing donors time to return

# Cox Penalized Regression model with LASSO Penalty

#-- two models after completed and after deferral
#keep only WB 

import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis

from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import FitFailedWarning
import sksurv.util
import warnings

from sksurv.preprocessing import OneHotEncoder


df=pd.read_csv("../1_data/private/preprocessed_data.csv")

mask = (df['Visit_yr'] >= 2017) 
df = df.loc[mask]

df_cox=df 
print(len(df))
print(df['DonorID'].nunique())

c:\Users\jagir\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


4286365
596176


In [13]:
print(list(df.columns))

['DonorID', 'Visit_Date', 'Visit_yr', 'Visit_Mo', 'donation_time', 'don_Sequence', 'col_Int', 'Visit_Age', 'sex', 'Fixed', 'MobileID', 'donation_product_x', 'DonProc', 'Outcome', 'ABO_RH', 'Disease_marker_results', 'HB_Value', 'race_group', 'race', 'Ferritin', 'category', 'DefCode', 'Def_start', 'Def_end', 'Deferral_permanent', 'EffectiveYr', 'EffectiveMo', 'OUTCOME_TYPE', 'CENSORED', 'time_to_return', 'Fixed_mobile', 'FIXED_NEXT', 'fixed_mobile_pattern', 'index', 'donation_product_y', 'first_time']


In [14]:
# convert columns to categorical varibles
cols = ['sex', 'OUTCOME_TYPE']
df_cox[cols] = df_cox[cols].astype('category')

df_cox = pd.get_dummies(df_cox, prefix=cols, dummy_na=True, columns = cols)

print(list(df_cox.columns))

# 'Visit_yr', 'Visit_Mo', 'race_group', 'Ferritin',  'HB_Value', 'donation_product',
covariates = ['Visit_Age', 'sex_F', 'sex_M', 'Fixed_mobile', 'OUTCOME_TYPE_completed', 'OUTCOME_TYPE_low hgb', 'OUTCOME_TYPE_no draw', 'OUTCOME_TYPE_other deferral']

#- so deferral period is a covariate
#- deferral is a covariate
#- index donation is a covarite (M,F) 
#- add one one column for idi




['DonorID', 'Visit_Date', 'Visit_yr', 'Visit_Mo', 'donation_time', 'don_Sequence', 'col_Int', 'Visit_Age', 'Fixed', 'MobileID', 'donation_product_x', 'DonProc', 'Outcome', 'ABO_RH', 'Disease_marker_results', 'HB_Value', 'race_group', 'race', 'Ferritin', 'category', 'DefCode', 'Def_start', 'Def_end', 'Deferral_permanent', 'EffectiveYr', 'EffectiveMo', 'CENSORED', 'time_to_return', 'Fixed_mobile', 'FIXED_NEXT', 'fixed_mobile_pattern', 'index', 'donation_product_y', 'first_time', 'sex_F', 'sex_M', 'sex_nan', 'OUTCOME_TYPE_completed', 'OUTCOME_TYPE_low hgb', 'OUTCOME_TYPE_no draw', 'OUTCOME_TYPE_other deferral', 'OUTCOME_TYPE_nan']


In [4]:
print(list(df_cox.columns))

['DonorID', 'Visit_Date', 'Visit_yr', 'Visit_Mo', 'donation_time', 'don_Sequence', 'col_Int', 'Visit_Age', 'Fixed', 'MobileID', 'donation_product_x', 'DonProc', 'Outcome', 'ABO_RH', 'Disease_marker_results', 'HB_Value', 'race_group', 'race', 'Ferritin', 'category', 'DefCode', 'Def_start', 'Def_end', 'Deferral_permanent', 'EffectiveYr', 'EffectiveMo', 'CENSORED', 'time_to_return', 'Fixed_mobile', 'FIXED_NEXT', 'fixed_mobile_pattern', 'index', 'donation_product_y', 'first_time', 'sex_F', 'sex_M', 'sex_nan', 'OUTCOME_TYPE_completed', 'OUTCOME_TYPE_low hgb', 'OUTCOME_TYPE_no draw', 'OUTCOME_TYPE_other deferral', 'OUTCOME_TYPE_nan']


In [5]:
#helper functions to make plots

#a function to plot coefficients 
def plot_coef(coefs, n_highlight):
    _, ax = plt.subplots(figsize=(9, 6))
    n_features = coefs.shape[0]
    alphas = coefs.columns
    for row in coefs.itertuples():
        ax.semilogx(alphas, row[1:], ".-", label=row.Index)
        
    alpha_min = alphas.min()
    top_coefs = coefs.loc[:, alpha_min].map(abs).sort_values().tail(n_highlight)
    for name in top_coefs.index:
        coef = coefs.loc[name, alpha_min]
        plt.text(
            alpha_min, coef, name + "   ",
            horizontalalignment="right",
            verticalalignment="center"
        )

    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.grid(True)
    ax.set_xlabel("alpha")
    ax.set_ylabel("coefficient")
    
#plot mean concordance index and std deviation for each alpa across all folds
def plot_mean_concordance_index(alphas, mean, std, gridCV):
    fig, ax = plt.subplots(figsize=(9, 6))
    ax.plot(alphas, mean)
    ax.fill_between(alphas, mean - std, mean + std, alpha=.15)
    ax.set_xscale("log")
    ax.set_ylabel("concordance index")
    ax.set_xlabel("alpha")
    ax.axvline(gridCV.best_params_["coxnetsurvivalanalysis__alphas"][0], c="C1")
    ax.axhline(0.5, color="grey", linestyle="--")
    ax.grid(True)
    
#plot coefficients to be used
def plot_coef_to_use(non_zero_coefs, coef_order):
    _, ax = plt.subplots(figsize=(6, 8))
    non_zero_coefs.loc[coef_order].plot.barh(ax=ax, legend=False)
    ax.set_xlabel("coefficient")
    ax.grid(True)

In [6]:
#helper function for feature selection

def feature_selection(cox_lasso, X_train, Y_train):
    
    #generate and plot coeffecients to choose best aplha
    coef_lasso = pd.DataFrame(
        cox_lasso.coef_,
        index=X_train.columns,
        columns=np.round(cox_lasso.alphas_, 5)
    )
    
    

    plot_coef(coef_lasso, n_highlight=5)
    
    
    #choosing the right aplha
    pipe = make_pipeline(
        StandardScaler(),
        CoxnetSurvivalAnalysis(l1_ratio=0.9, alpha_min_ratio=0.01, max_iter=100)
    )
    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)
    pipe.fit(X_train, Y_train)
    
    #performing 5 fold cross validation
    estimated_alphas = pipe.named_steps["coxnetsurvivalanalysis"].alphas_
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    gridCV = GridSearchCV(
            make_pipeline(StandardScaler(), CoxnetSurvivalAnalysis(l1_ratio=0.9)),
            param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in estimated_alphas]},
            cv=cv,
            error_score=0.5,
            n_jobs=4).fit(X_train, Y_train)

    results = pd.DataFrame(gridCV.cv_results_)
    
    #plot mean_concordance index
    alphas = results.param_coxnetsurvivalanalysis__alphas.map(lambda x: x[0])
    mean = results.mean_test_score
    std = results.std_test_score
    
    plot_mean_concordance_index(alphas, mean, std, gridCV)
    
    #choose best model, 
    best_model = gridCV.best_estimator_.named_steps["coxnetsurvivalanalysis"]
    best_coefs = pd.DataFrame(
        best_model.coef_,
        index=X_train.columns,
        columns=["coefficient"]
    )

    #number of coefficients that are not zero
    non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
    print("Number of non-zero coefficients: {}".format(non_zero))

    
    #create a list of coefs to be used
    non_zero_coefs = best_coefs.query("coefficient != 0")
    coef_order = non_zero_coefs.abs().sort_values("coefficient").index
    
    #plot_coefficients to be used
    plot_coef_to_use(non_zero_coefs, coef_order)
    
    return gridCV


In [7]:
##df_cox.fillna(999, inplace=True)
#df_cox.replace([np.inf, -np.inf], np.nan, inplace=True)


In [13]:
#model fitting
#df_cox_def = df_cox.query("DEF_TYPE == 'low hgb'")
#X = df_cox.loc[:, covariates]


x_df=df_cox[['sex', 'OUTCOME_TYPE', "Fixed_mobile", 'race_group']].astype('category')

X = OneHotEncoder().fit_transform(x_df)
X.head()

#X= df_cox.reindex(columns=covariates)

   #Structured array
Y= sksurv.util.Surv.from_dataframe('CENSORED', 'time_to_return', df_cox)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.90, random_state=42)

#fit the model
#cox_lasso = CoxnetSurvivalAnalysis(l1_ratio=1.0, alpha_min_ratio=0.01)
cox_lasso = CoxPHSurvivalAnalysis()
cox_lasso.fit(X_train, Y_train)




CoxPHSurvivalAnalysis()

In [14]:
print(pd.Series(cox_lasso.coef_, index=X_train.columns))
#cox_lasso.coef_


sex=M                          0.157304
OUTCOME_TYPE=low hgb          -0.439231
OUTCOME_TYPE=no draw          -0.175705
OUTCOME_TYPE=other deferral   -0.267083
Fixed_mobile=1                 0.468717
race_group=2                  -0.078720
race_group=3                  -0.014800
race_group=4                  -0.019882
race_group=1                  -0.165779
race_group=2                  -0.366405
race_group=3                  -0.183603
race_group=4                  -0.287973
race_group=U                  -0.337439
dtype: float64


In [ ]:
gcv = feature_selection(cox_lasso, X_train, Y_train)


In [ ]:
#prediction

#fit test set to perfrom prediction of hazard function and survival function
coxnet_pred = make_pipeline(
    StandardScaler(),
    CoxnetSurvivalAnalysis(l1_ratio=0.9, fit_baseline_model=True)
)
coxnet_pred.set_params(**gcv.best_params_)
coxnet_pred.fit(X_test, Y_test)

surv_fns = coxnet_pred.predict_survival_function(X_test)
